## Goroutine là gì?

Vài tính chát quan trọng:
- Lightweight: nhẹ, chỉ chiếm 2KB stack khi khởi tạo
- Growable: stack này có thể dễ dàng tăng size hoặc giảm size khi cần thiết
- Multiplexed: Rất nhiều goroutine có thể chạy chỉ trên vài OS Threads
- Cheap: Có thể tạo ra hàng triệu goroutines một cách nhẹ nhàng

## The "Boring" Function Example

Đây là ví dụ của Rob Pike, gọi là Boring function, nó in ra boring mỗi 1s

In [4]:
package main

import (
    "fmt"
    "time"
)

func boring(msg string) {
    for i := 0; ; i++ {
        fmt.Printf("%s %d\n", msg, i)
        time.Sleep(time.Second)
    }
}

func main() {
    boring("boring!")
}

boring! 0
boring! 1
boring! 2
boring! 3
boring! 4
boring! 5
boring! 6
boring! 7
boring! 8
boring! 9
boring! 10


signal: interrupt


Nếu chạy thử hàm này sẽ in ra boring! 0, boring! 1, boring! 2, boring! 3... tới vô tận

## Adding the Magic: The go Statement

Chỉ cần thêm từ khóa `go` trước 1 function, sẽ biến func thành goroutine

In [10]:
func boring(msg string) {
    for i := 0; ; i++ {
        fmt.Printf("%s %d\n", msg, i)
        time.Sleep(time.Second)
    }
}

func main() {
    go boring("boring!")
    fmt.Println("I'm listening.")
    time.Sleep(2 * time.Second)
    fmt.Println("You're boring; I'm leaving.")
}

I'm listening.
boring! 0
boring! 1
You're boring; I'm leaving.


#### Điểm lưu ý đầu tiên: 
- Hàm main cũng là một goroutine, nó sẽ chạy đầu tiên
- Khi hàm main kết thúc, nó sẽ dừng (terminate) `toàn bộ` các goroutine đang chạy

#### Điểm lưu ý tiếp theo:
- Sau khi hàm main chạy, nó sẽ in ra I'm listening
- Sau đó main và boring chạy dồng thời (concurrently)
- Hàm main ko làm gì, sleep trong 2s, hàm boring cũng chạy song song, in ra được boring! 0, 1, 2
- Sau 2s, hàm main in ra You're boring; I'm leaving. và kết thúc, hàm boring cũng kết thúc tức thì, không in thêm ra được gì nữa

#### Về Race condition
- Đôi khi chúng ta sẽ thấy func chỉ in ra tới boring! 1, nhưng đôi khi thấy func in ra tới boring! 2
- Goroutine phải được scheduled, nên cần thời gian chứ không phải liền tức thì
- Goroutine của hàm boring sẽ ngủ đông 1s rồi được gọi dậy, của hàm main thì 2s
- Giả sử tại thời điểm 0s:
    - main được khởi tạo in ra I'm listening.
    - boring được khởi tạo ngay sau và in ra boring! 0
- Sau 1s thì boring được gọi dậy in ra boring! 1, main vẫn ngủ đông
- Sau 2s thì boring lẫn main được gọi dậy
    - Lúc này thì main lẫn boring là bình đẳng, schedule gọi func nào dậy trước sẽ thực hiện func đó trước
    - Nếu boring trước, sẽ in ra boring! 2, sau đó main được gọi dậy in ra You're boring; I'm leaving. và toàn bộ goroutine kết thúc
    - Nếu main trước thì main được gọi dậy in ra You're boring; I'm leaving. và toàn bộ goroutine kết thúc, boring sẽ không in ra được boring! 2

## Important Concepts

#### Goroutine Lifecycle  
1. Created - goroutine is spawned  
2. Running - actively executing  
3. Sleeping - time.Sleep() or waiting  
4. Terminated - function returns  

## Common Mistakes

### 1. Forgetting to Wait

In [11]:
// Wrong - program exits immediately
func main() {
    go boring("boring!")
    // No wait - program terminates
}

Nếu không có gì ngừng hàm main lại, thì hàm main sẽ chạy và kết thúc (terminate) ngay lập tức, boring cũng bị dừng ngay tức khắc mà không in ra được gì  
Có kĩ thuật gọi là sync.WaitGroup, hãy xem phần 09.

### 2. Not Understanding Concurrency

In [ ]:
// This doesn't make them run "in order"
go func1()
go func2()
// They run concurrently, not sequentially!

Nhớ để ý func1 và func2 chạy đồng thời, ko theo tuần tự là được, ví dụ như ở dưới, có lúc in boring1 trước, có lúc in boring2 trước.  
Chạy đoạn code bên dưới nhiều lần để cảm nhận concurrency.

In [11]:
func boring(msg string) {
    for i := 0; ; i++ {
        fmt.Printf("%s %d\n", msg, i)
        time.Sleep(time.Second)
        fmt.Println()
    }
}

func main() {
    go boring("boring 1!")
    go boring("boring 2!")
    fmt.Println("I'm listening.")
    time.Sleep(5 * time.Second)
    fmt.Println("You're boring; I'm leaving.")
}

I'm listening.
boring 2! 0
boring 1! 0

boring 2! 1

boring 1! 1


boring 1! 2
boring 2! 2

boring 2! 3

boring 1! 3

boring 1! 4

boring 2! 4
You're boring; I'm leaving.
